# The longest glider flight

**Problem**: Suppose you wanted to participate in a paper-airplane competition, and
you want to use what you know about the glider flight model to improve your
chances. For a given value of the aerodynamic efficiency, 
$R$, that you can obtain in your design, you
want to know what is the best initial velocity and launch angle to fly
the longest distance from a given height.

Recall that a glider motion is described by the following system of nonlinear ordinary differential equations.
\begin{align}
  \frac{\mathrm{d}v}{\mathrm{d}\tau} & = -\sin\theta - \frac{v^2}{R}, \\
  \frac{\mathrm{d}\theta}{\mathrm{d}\tau} & = - \frac{\cos\theta}{v} + v, \\
  \frac{\mathrm{d} x}{\mathrm{d}\tau} & = v \cos\theta, \\
  \frac{\mathrm{d} y}{\mathrm{d}\tau} & = v \sin\theta .
\end{align}
The equations contains a single parameter - the aerodynamic efficiency $R$ which is the ration of lift to drag forces acting on a glider. Here $\theta$ is the angle between the instantaneous direction of the velocity of the glider and the horizontal direction. The other parameters are the dimensionless time $\tau$, dimensionless speed of the glider $v$, and dimensionless cartesian coordinates of the glider in the vertical plane, $x$ and $y$. 

Load the required packages:

In [ ]:

using OrdinaryDiffEqTsit5
using PyPlot
using Optim

Implement the function calculating the right hand side of the equation of motion:

In [ ]:

"""
    glider!(dudt, u, p, t)

The system of differential equations for a glider fligt;
u = [v, theta, x, y]
"""
function glider!(dudt, u, p, _)
    # your code here
end

Implement a function that, given the initial conditions `u = [v0, theta0]`, solves the equation of motion from the launch until the landing. 

In [ ]:

"""
    sol = solve_eqs_motion_landing(u)

Solve the glider's equation of motion. Terminate the solver when
the glider lands, i.e. when y(t) = 0.
Here u[1] is the initial velocity and u[2] is the launch angle. 
"""
function solve_eqs_motion_landing(u)

    # Parameters
    global xmin, ymax, R

    # Initial conditions and ode parameters
    v0 = u[1]
    theta0 = u[2] 
    tspan = (0.0, 2000.0)              # huge upper limit, not going to be reached
    uu0 = [v0, theta0, xmin, ymax]     # initial conditions for the ODE solver

    # Solve ODE
    probl = ODEProblem(glider!, uu0, tspan, R)
    condition(u, _, _) =  # the condition for landing, your code here
    affect!(integrator) = terminate!(integrator)
    cb = ContinuousCallback(condition, affect!)
    sol = solve(probl, Tsit5(), callback=cb, abstol=1e-10, reltol=1e-10)

    return sol
end

Helper function:

In [ ]:

"""
    plot_trajectory(sol::ODESolution, label)

Given a solution of a glider equation of motion, sol, plot
the trajectory and print a graph label
"""
function plot_trajectory(sol::ODESolution, label)
    # Extract glider coordinates
    x = sol[3, :]
    y = sol[4, :]

    plot(x, y, label=label)
    xlabel("x")
    ylabel("y")
    grid(true)
    legend()
    
    return nothing
end

Parameters for the calculations:

In [ ]:

# Global parameters
const xmin = 0.0        # initial x
const ymax = 2.0        # initial hight
const R = 5.0;          # aerodynamic efficiency of the glider

Helper function:

Helper function:

In [ ]:
"""
    l = flight_distance(sol)

Given the solution of the equations of motion
of a glider, return the distance of the flight.
"""
function flight_distance(sol::ODESolution)
    return sol.u[end][3]
end

"""
    l = flight_negative_distance(u)

Given the initial velocity (u[1]) and the launch angle (u[2]),
solve the equations of motion of a glider and return the
*negative* distance of the flight.
"""
function flight_negative_distance(u)
    sol = solve_eqs_motion_landing(u)
    return -sol.u[end][3]
end

We are finding a minimum of a function - the largest distance of flight is given by the **minimum** of the negative of the distance.

Initial approximation for the minimization:

In [ ]:
initial_approx = [3.0, 0.0] 

For minimization, we are using `L-BFGS` that is a minimization method that requires a gradient.
We use automatic differentiation (instead of default finite differences) to calculate gradients,
by setting the keyword `autodiff` to `:forward`

In [ ]:
@time res = optimize(flight_negative_distance, initial_approx, LBFGS(); autodiff=:forward)

Define a helper function that wraps an angle to the range [-pi/2,pi/2)

In [ ]:
wrap_angle(phi) = mod(phi + pi/2, pi) - pi/2

Extract and print the optimal flight parameters:

In [ ]:
s = Optim.minimizer(res) 
s[2] = wrap_angle(s[2])
@info "optimal launch velocity:" round(s[1], digits=3)
@info "optimal launch angle (in radians):" round(s[2], digits=3)
@info "optimal launch angle (in degrees):" round(180 * s[2] / pi, digits=1)

Recalculate the best solution:

In [ ]:
sol = solve_eqs_motion_landing(s);

Flight distance:

In [ ]:
lflight = flight_distance(sol);
@info "flight distance:" round(lflight, digits=3)

Plot the longest trajectory and, for a comparison, an arbitrary shorter one:

In [ ]:
figure(figsize=(9,6))
plot_trajectory(sol, "The longest trajectory")
# For comparison, plot a shorter trajectory
sol2 = solve_eqs_motion_landing(initial_approx);
plot_trajectory(sol2, "A shorter trajectory")